In [33]:
import numpy as np
import tensorflow as tf
from tensorflow.python.layers.core import Dense
tf.__version__

'1.2.0'

In [8]:
# https://github.com/tensorflow/tensorflow/issues/10815

In [9]:
n_steps = 14
n_input = 2
n_output = 1

In [10]:
def get_batch(batch_size):
    x = np.random.uniform(0, 1, size=[batch_size,n_steps, n_input])
    y = np.flip(x, axis=1)
    y = np.sum(y, axis=2)
    y = y.reshape((batch_size, n_steps, 1))
    
    seq = np.empty((batch_size), dtype=np.int)
    seq.fill(n_steps)
    return x, y, seq

x, y, seq = get_batch(100)

In [38]:
n_steps = 14
n_input = 2
n_output = 1
inp_seq_len = out_seq_len = n_steps
layers_stacked_count = 2  # Number of stacked recurrent cells, on the neural depth axis. 
n_hidden = 20

tf.reset_default_graph()

sess = tf.InteractiveSession()

# Placeholders
enc_inp = tf.placeholder(tf.float32, [None, inp_seq_len, n_input], name='encoder_input')
dec_target = tf.placeholder(tf.float32, [None, out_seq_len, n_output], name ='decoder_input')
#dec_targets = [tf.placeholder(tf.float32, [None, n_output]) for i in range(out_seq_len)]

target_length = tf.placeholder(tf.int32, [None], name='target_seq_length')
keep_prob = tf.placeholder(tf.float32, [], name='dropout_keep_prob')
sample_rate = tf.placeholder(tf.float32, [], name = 'sample_rate')

# ---- Encoder
enc_cells = [tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicLSTMCell(n_hidden), output_keep_prob=keep_prob) for i in range(layers_stacked_count)]
enc_stk_cell = tf.contrib.rnn.MultiRNNCell(enc_cells)

encoded_outputs, encoded_states = tf.nn.dynamic_rnn(enc_stk_cell, enc_inp, dtype=tf.float32)

# ---- Decoder
dec_cells = [tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicLSTMCell(n_hidden), output_keep_prob=keep_prob) for i in range(layers_stacked_count)]
dec_stk_cell = tf.contrib.rnn.MultiRNNCell(dec_cells)

#helper = tf.contrib.seq2seq.TrainingHelper(expect, expect_length) # Old
#n_in_layer = tf.layers.dense()
hlay = Dense(n_output, dtype=tf.float32)
print(type(hlay))
helper = tf.contrib.seq2seq.ScheduledOutputTrainingHelper(dec_target, target_length, sample_rate, next_input_layer=hlay)

decoder = tf.contrib.seq2seq.BasicDecoder(cell=dec_stk_cell, helper=helper, initial_state=encoded_states)

decoder_outputs, final_decoder_state, length = tf.contrib.seq2seq.dynamic_decode(decoder)
decoder_logits = decoder_outputs.rnn_output

h = tf.contrib.layers.fully_connected(decoder_logits, n_output)

diff = tf.squared_difference(h, dec_target)
batch_loss = tf.reduce_sum(diff, axis=1)
loss = tf.reduce_mean(batch_loss)

optimiser = tf.train.AdamOptimizer(1e-3)
training_op = optimiser.minimize(loss)

init_op = tf.global_variables_initializer()
    

<class 'tensorflow.python.layers.core.Dense'>


In [39]:
sess = tf.InteractiveSession()

init_op.run()

for e in range(10):
    for i in range(100):
        batch_x, batch_y ,seq = get_batch(100)
        training_op.run(feed_dict={enc_inp:batch_x, dec_target: batch_y, target_length:seq, keep_prob:0.5, sample_rate:0.5})
        
    print(loss.eval(feed_dict={enc_inp:batch_x, dec_target: batch_y, target_length:seq, keep_prob:1,sample_rate:0}))

2.1538
1.93195
1.86243
1.22235
1.02921
0.855889
0.733073
0.684626
0.629023
0.498449


In [32]:
test_x, test_y, seq = get_batch(2)
print(test_y[0])
print('---------')
feed_dict = {enc_inp:test_x, target_length:seq, keep_prob:1, sample_rate:0}
pred = h.eval(feed_dict=feed_dict)

print(test_y[0]-pred[0])



[[ 0.98417242]
 [ 0.62155374]
 [ 0.52138793]
 [ 1.16872927]
 [ 1.03178255]
 [ 1.45997571]
 [ 1.27879388]
 [ 0.64727848]
 [ 0.74829998]
 [ 1.11627821]
 [ 0.22984916]
 [ 0.83748326]
 [ 1.14607962]
 [ 1.30477196]]
---------


InvalidArgumentError: Shape [-1,14,1] has negative dimensions
	 [[Node: decoder_input = Placeholder[dtype=DT_FLOAT, shape=[?,14,1], _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'decoder_input', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ppyht2/.local/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/ppyht2/.local/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ppyht2/.local/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/ppyht2/.local/lib/python3.5/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/home/ppyht2/.local/lib/python3.5/site-p

In [16]:
import tensorflow as tf
with tf.Graph().as_default():
    batch_size = 32
    nsteps = 100
    ndims = 5
    sequence_length = [nsteps] * batch_size
    sampling_probability = 0.5
    num_units = 20

    cell = tf.contrib.rnn.BasicRNNCell(
        num_units,
    )

    inputs = tf.random_uniform((batch_size, nsteps, ndims))

    output, state = tf.nn.dynamic_rnn(
        cell,
        inputs,
        dtype=tf.float32,
    )

    cell = tf.contrib.rnn.BasicRNNCell(
        num_units,
    )

    helper = tf.contrib.seq2seq.ScheduledOutputTrainingHelper(
        output,
        sequence_length,
        sampling_probability,
    )

    initial_state = tf.zeros((batch_size, num_units))
    decoder = tf.contrib.seq2seq.BasicDecoder(
        cell,
        helper,
        initial_state,
    )

    decoded = tf.contrib.seq2seq.dynamic_decode(
        decoder,
    )

    run_ops = {'decoded': decoded}
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        ret = sess.run(run_ops)

InvalidArgumentError: TensorArray dtype is int32 but Op is trying to write dtype bool.
	 [[Node: decoder/while/TensorArrayWrite_1/TensorArrayWriteV3 = TensorArrayWriteV3[T=DT_BOOL, _class=["loc:@decoder/while/BasicDecoderStep/ScheduledOutputTrainingHelperSample/Cast"], _device="/job:localhost/replica:0/task:0/cpu:0"](decoder/while/TensorArrayWrite_1/TensorArrayWriteV3/Enter, decoder/while/Identity/_55, decoder/while/BasicDecoderStep/ScheduledOutputTrainingHelperSample/Cast, decoder/while/Identity_2/_57)]]

Caused by op 'decoder/while/TensorArrayWrite_1/TensorArrayWriteV3', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ppyht2/.local/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/ppyht2/.local/lib/python3.5/site-packages/traitlets/config/application.py", li